<a href="https://colab.research.google.com/github/ggsgranados/TelecomX-LATAM-Parte-2/blob/main/Telecom_X_Parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports necesarios

In [12]:
import pandas as pd

##📌 Extracción del archivo tratado

In [13]:
# Importando el JSON y convirtiendolo a un DataFrame
df = pd.read_csv('https://raw.githubusercontent.com/ggsgranados/TelecomX-LATAM-Parte-2/refs/heads/main/df_normalizado.csv')

In [14]:
df.head(5)

,Churn,SeniorCitizen,tenure,Charges.Monthly,Charges.Total,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Cuentas_Diarias,Churn_Label
0,0.0,0,9,65.6,593.30,1,0,0,1,0,...,1,0,0,1,0,0,0,1,2.155059,Permaneció
1,0.0,0,9,59.9,542.40,0,1,1,0,1,...,0,0,1,0,0,0,0,1,1.967806,Permaneció
2,1.0,0,4,73.9,280.85,0,1,1,0,1,...,0,0,0,1,0,0,1,0,2.427727,Se dio de baja
3,1.0,1,13,98.0,1237.85,0,1,0,1,1,...,0,0,0,1,0,0,1,0,3.219448,Se dio de baja
4,1.0,1,3,83.9,267.40,1,0,0,1,1,...,0,0,0,1,0,0,0,1,2.756242,Se dio de baja


##❌ Eliminación de columnas irrelevantes

In [15]:
def remove_negative_dummies(df):
    columns_to_drop = []
    for col in df.columns:
        # Buscar columnas que terminen en '_No' o '_no'
        if col.endswith('_No') or col.endswith('_no'):
            # Identificar la columna correspondiente con lógica positiva
            positive_col_name_yes = col.replace('_No', '_Yes').replace('_no', '_yes')
            positive_col_name_blank = col.replace('_No', '_ ').replace('_no', '_ ')
            positive_col_name_single = col.replace('_No', '').replace('_no', '')

            # Verificar si la columna de lógica positiva existe
            if positive_col_name_yes in df.columns or positive_col_name_blank in df.columns or positive_col_name_single in df.columns:
                columns_to_drop.append(col)

    if columns_to_drop:
        print(f"\nEliminando las siguientes columnas con lógica negativa: {columns_to_drop}")
        df = df.drop(columns=columns_to_drop)
    else:
        print("\nNo se encontraron columnas con lógica negativa ('_No' o '_no') para eliminar.")

    return df

# Aplicar la función al DataFrame
df_normalizado = remove_negative_dummies(df)

print("\nColumnas después de eliminar las de lógica negativa:")
print(df_normalizado.columns.tolist())
print("\nEstado final del DataFrame:")
print(df_normalizado.info())


Eliminando las siguientes columnas con lógica negativa: ['Partner_No', 'Dependents_No', 'PhoneService_No', 'MultipleLines_No', 'OnlineSecurity_No', 'OnlineBackup_No', 'DeviceProtection_No', 'TechSupport_No', 'StreamingTV_No', 'StreamingMovies_No', 'PaperlessBilling_No']

Columnas después de eliminar las de lógica negativa:
['Churn', 'SeniorCitizen', 'tenure', 'Charges.Monthly', 'Charges.Total', 'gender_Female', 'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No internet service', 'OnlineSecurity_Yes', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No internet service', 'DeviceProtection_Yes', 'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No internet service', 'StreamingTV_Yes', 'StreamingMovies_No internet service', 'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contrac

In [16]:
# Verificar si la columna 'gender_Female' existe antes de intentar eliminarla
if 'gender_Female' in df_normalizado.columns:
    # Eliminar la columna 'gender_Female'
    df_normalizado = df_normalizado.drop(columns=['gender_Female'])
    print("\nLa columna 'gender_Female' ha sido eliminada.")
else:
    print("\nLa columna 'gender_Female' no se encontró en el DataFrame.")

# Imprimir las columnas actualizadas
print("\nColumnas después de eliminar 'gender_Female':")
print(df_normalizado.columns.tolist())

# Imprimir el estado final del DataFrame
print("\nEstado final del DataFrame:")
print(df_normalizado.info())



La columna 'gender_Female' ha sido eliminada.

Columnas después de eliminar 'gender_Female':
['Churn', 'SeniorCitizen', 'tenure', 'Charges.Monthly', 'Charges.Total', 'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No internet service', 'OnlineSecurity_Yes', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No internet service', 'DeviceProtection_Yes', 'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No internet service', 'StreamingTV_Yes', 'StreamingMovies_No internet service', 'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes', 'PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check', 'Cuentas_Diarias', 'Churn_Label']

Estado f

## ✅ Verificación de la proporción de cancelación (Churn)

In [19]:
churn_proportion = df_normalizado['Churn'].value_counts(normalize=True)
print("\nProporción de clientes que cancelaron (Churn = 1) vs. activos (Churn = 0) :")
churn_proportion



Proporción de clientes que cancelaron (Churn = 1) vs. activos (Churn = 0) :


,proportion
Churn,
0.0,0.73463
1.0,0.26537
